In [ ]:
# to run this create new env with PySpice lib at Python 3.10
# or you can fuck it and just use generated JSONs in spice folder
import json
import numpy as np
from pathlib import Path
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

In [ ]:
def save_to_file(path: Path, freqs: np.ndarray, gains: np.ndarray) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.touch()
    with path.open("w") as file:
        freqs = [float(x) for x in freqs]
        gains = [float(x) for x in gains]
        json.dump({"freqs": freqs, "gains": gains}, file)

In [ ]:
circuit = Circuit("Bassman tone stack")

circuit.SinusoidalVoltageSource(1, "v_in", circuit.gnd, amplitude=1@u_V)
circuit.C(1, "v_in", "1", 0.25@u_nF)
circuit.R(1, "1", "out", 125.0@u_kΩ)
circuit.R(2, "out", "4", 125.0@u_kΩ)
circuit.R(3, "v_in", "3", 56.0@u_kΩ)
circuit.C(2, "3", "4", 20.0@u_nF)
circuit.C(3, "3", "6", 20.0@u_nF)
circuit.R(4, "4", "5", 0.5@u_MΩ)
circuit.R(5, "5", "6", 12.5@u_kΩ)
circuit.R(6, "6", circuit.gnd, 12.5@u_kΩ)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
# simulate with transfering chirp signal
analysis = simulator.ac(start_frequency=1@u_Hz, stop_frequency=10@u_kHz, number_of_points=100,  variation="dec")
out = np.array(analysis.out)
save_to_file(Path("./spice/bassman.json"), analysis.frequency, 20 * np.log10(np.abs(out)))

In [ ]:
circuit = Circuit("Double-pole RLC lowpass")

circuit.SinusoidalVoltageSource(1, "v_in", circuit.gnd, amplitude=1@u_V)
circuit.R(1, "v_in", "2", 25@u_Ω)
circuit.L(1, "2", "out", 10@u_mH)
circuit.C(1, "out", circuit.gnd, 1@u_uF)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
# simulate with transfering chirp signal
analysis = simulator.ac(start_frequency=1@u_Hz, stop_frequency=10@u_kHz, number_of_points=100,  variation="dec")
out = np.array(analysis.out)
save_to_file(Path("./spice/rlc_lowpass.json"), analysis.frequency, 20 * np.log10(np.abs(out)))

In [ ]:
circuit = Circuit("Bridged-T bandstop")

circuit.SinusoidalVoltageSource(1, "v_in", circuit.gnd, amplitude=1@u_V)
circuit.R(1, circuit.gnd, "2", 53.8@u_kΩ)
circuit.R(2, "v_in", "out", 1@u_MΩ)
circuit.C(1, "v_in", "2", 15@u_nF)
circuit.C(2, "2", "out", 15@u_nF)
circuit.R(3, circuit.gnd, "out", 1@u_MΩ)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
# simulate with transfering chirp signal
analysis = simulator.ac(start_frequency=1@u_Hz, stop_frequency=10@u_kHz, number_of_points=100,  variation="dec")
out = np.array(analysis.out)
save_to_file(Path("./spice/bridgedt_bandstop.json"), analysis.frequency, 20 * np.log10(np.abs(out)))